In [1]:
import pandas as pd

df = pd.read_csv("../data/Nazario_5.csv")
df.head()

ModuleNotFoundError: No module named 'pandas'

In [2]:
!pip install pandas


zsh:1: command not found: pip


In [3]:
!python3 -m ensurepip --upgrade


Defaulting to user installation because normal site-packages is not writeable
Looking in links: /var/folders/jc/jdzvkhjx49g2lr0v427m8qvm0000gn/T/tmpt9k8qpcu


In [4]:
!python3 -m pip install pandas


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 10.8 MB 1.4 MB/s eta 0:00:01
     |████████████████████████████████| 5.3 MB 17.9 MB/s eta 0:00:01
     |████████████████████████████████| 509 kB 39.4 MB/s eta 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [5]:
import pandas as pd

df = pd.read_csv("../data/Nazario_5.csv")
df.head()

,sender,receiver,date,subject,body,label,urls
0,"""Hu, Sylvia"" <Sylvia.Hu@ENRON.com>","""Acevedo, Felecia"" <Felecia.Acevedo@ENRON.com>...","Fri, 29 Jun 2001 08:36:09 -0500","FW: June 29 -- BNA, Inc. Daily Labor Report",User ID: enrondlr\nPW: bnaweb22\n\n\n ...,0,"['http://web.bna.com', 'http://pubs.bna.com/ip..."
1,"""Webb, Jay"" <Jay.Webb@ENRON.com>","""Lambie, Chris"" <Chris.Lambie@ENRON.com>","Fri, 29 Jun 2001 09:37:04 -0500",NGX failover plan.,"\nHi Chris, \n\nTonight we are rolling out a ...",0,[]
2,"""Symms, Mark"" <Mark.Symms@ENRON.com>","""Thomas, Paul D."" <Paul.D.Thomas@ENRON.com>","Fri, 29 Jun 2001 08:39:30 -0500",RE: Intranet Site,Rika r these new?\n\n -----Original Message---...,0,['http://eastpower.dev.corp.enron.com/summary/...
3,"""Thorne, Judy"" <Judy.Thorne@ENRON.com>","""Grass, John"" <John.Grass@ENRON.com>, ""Nemec, ...","Fri, 29 Jun 2001 10:35:17 -0500",FW: ENA Upstream Company information,"John/Gerald,\n\nWe are currently trading under...",0,[]
4,"""Williams, Jason R (Credit)"" <Jason.R.Williams...","""Nemec, Gerald"" <Gerald.Nemec@ENRON.com>, ""Dic...","Fri, 29 Jun 2001 10:40:02 -0500",New Master Physical,Gerald and Stacy -\n\nAttached is a worksheet ...,0,[]


In [6]:
def count_urls(url_string):
    try:
        urls = eval(url_string)   # convert "['http://...']" into a Python list
        return len(urls)
    except:
        return 0

df["num_urls"] = df["urls"].apply(count_urls)
df[["urls", "num_urls"]].head()

,urls,num_urls
0,"['http://web.bna.com', 'http://pubs.bna.com/ip...",41
1,[],0
2,['http://eastpower.dev.corp.enron.com/summary/...,3
3,[],0
4,[],0


In [7]:
import re

def extract_domain(sender):
    # sender looks like:  '"Hu, Sylvia" <Sylvia.Hu@ENRON.com>'
    match = re.search(r'@([^>]+)>?', str(sender))
    if match:
        return match.group(1).lower()
    return None

df["sender_domain"] = df["sender"].apply(extract_domain)
df[["sender", "sender_domain"]].head()

,sender,sender_domain
0,"""Hu, Sylvia"" <Sylvia.Hu@ENRON.com>",enron.com
1,"""Webb, Jay"" <Jay.Webb@ENRON.com>",enron.com
2,"""Symms, Mark"" <Mark.Symms@ENRON.com>",enron.com
3,"""Thorne, Judy"" <Judy.Thorne@ENRON.com>",enron.com
4,"""Williams, Jason R (Credit)"" <Jason.R.Williams...",enron.com


In [8]:
def extract_tld(domain):
    if domain is None:
        return None
    parts = domain.split('.')
    # handle domains like "co.uk"
    if len(parts) >= 2:
        return parts[-1]  # last part
    return None

df["tld"] = df["sender_domain"].apply(extract_tld)
df[["sender_domain", "tld"]].head()

,sender_domain,tld
0,enron.com,com
1,enron.com,com
2,enron.com,com
3,enron.com,com
4,enron.com,com


In [9]:
suspicious_tlds = {
    "ru", "cn", "br", "xyz", "top", "click", "link", "rest",
    "monster", "buzz", "tk", "to", "ga", "ml", "cf"
}

def is_suspicious(tld):
    if tld is None:
        return 0
    return 1 if tld in suspicious_tlds else 0

df["suspicious_tld"] = df["tld"].apply(is_suspicious)
df[["sender_domain", "tld", "suspicious_tld"]].head()

,sender_domain,tld,suspicious_tld
0,enron.com,com,0
1,enron.com,com,0
2,enron.com,com,0
3,enron.com,com,0
4,enron.com,com,0


In [10]:
def extract_display_name(sender):
    # Example: '"Hu, Sylvia" <Sylvia.Hu@ENRON.com>'
    if sender is None:
        return None
    sender = str(sender)
    if "<" in sender:
        return sender.split("<")[0].strip().strip('"').lower()
    return sender.lower()

df["display_name"] = df["sender"].apply(extract_display_name)
df[["sender", "display_name"]].head()

,sender,display_name
0,"""Hu, Sylvia"" <Sylvia.Hu@ENRON.com>","hu, sylvia"
1,"""Webb, Jay"" <Jay.Webb@ENRON.com>","webb, jay"
2,"""Symms, Mark"" <Mark.Symms@ENRON.com>","symms, mark"
3,"""Thorne, Judy"" <Judy.Thorne@ENRON.com>","thorne, judy"
4,"""Williams, Jason R (Credit)"" <Jason.R.Williams...","williams, jason r (credit)"


In [11]:
# Common brand names targeted by phishing
brand_keywords = [
    "amazon", "paypal", "apple", "microsoft", "netflix",
    "bank", "chase", "wells fargo", "boa", "bank of america",
    "dhl", "ups", "fedex"
]

def brand_domain_mismatch(row):
    name = row["display_name"]
    domain = row["sender_domain"]

    if name is None or domain is None:
        return 0

    # check if display name contains a brand keyword
    for brand in brand_keywords:
        if brand in name:
            # check if domain does NOT contain the brand
            if brand not in domain:
                return 1   # suspicious mismatch
    return 0

df["brand_mismatch"] = df.apply(brand_domain_mismatch, axis=1)
df[["display_name", "sender_domain", "brand_mismatch"]].head()

,display_name,sender_domain,brand_mismatch
0,"hu, sylvia",enron.com,0
1,"webb, jay",enron.com,0
2,"symms, mark",enron.com,0
3,"thorne, judy",enron.com,0
4,"williams, jason r (credit)",enron.com,0


In [12]:
def compute_phishing_score(row):
    score = 0
    
    # suspicious TLD
    score += row["suspicious_tld"]
    
    # brand/domain mismatch
    score += row["brand_mismatch"]
    
    # too many URLs in body
    if row["num_urls"] > 5:
        score += 1
    
    return score

df["phishing_score"] = df.apply(compute_phishing_score, axis=1)
df[["num_urls", "suspicious_tld", "brand_mismatch", "phishing_score"]].head()

,num_urls,suspicious_tld,brand_mismatch,phishing_score
0,41,0,0,1
1,0,0,0,0
2,3,0,0,0
3,0,0,0,0
4,0,0,0,0


In [13]:
df[["num_urls", "suspicious_tld", "brand_mismatch", "phishing_score"]].describe()

,num_urls,suspicious_tld,brand_mismatch,phishing_score
count,3065.000000,3065.000000,3065.000000,3065.000000
mean,1.050897,0.020228,0.064274,0.107015
std,9.702091,0.140804,0.245281,0.316485
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000
max,467.000000,1.000000,1.000000,2.000000


In [14]:
import pandas as pd
import numpy as np
import re

df = pd.read_csv("./data/Nazario_5.csv")
df.head()

FileNotFoundError: [Errno 2] No such file or directory: './data/Nazario_5.csv'

In [15]:
!pwd

/Users/inesnix/commSec/final_project/notebooks


In [16]:
import pandas as pd
import numpy as np
import re

df = pd.read_csv("../data/Nazario_5.csv")
df.head()

,sender,receiver,date,subject,body,label,urls
0,"""Hu, Sylvia"" <Sylvia.Hu@ENRON.com>","""Acevedo, Felecia"" <Felecia.Acevedo@ENRON.com>...","Fri, 29 Jun 2001 08:36:09 -0500","FW: June 29 -- BNA, Inc. Daily Labor Report",User ID: enrondlr\nPW: bnaweb22\n\n\n ...,0,"['http://web.bna.com', 'http://pubs.bna.com/ip..."
1,"""Webb, Jay"" <Jay.Webb@ENRON.com>","""Lambie, Chris"" <Chris.Lambie@ENRON.com>","Fri, 29 Jun 2001 09:37:04 -0500",NGX failover plan.,"\nHi Chris, \n\nTonight we are rolling out a ...",0,[]
2,"""Symms, Mark"" <Mark.Symms@ENRON.com>","""Thomas, Paul D."" <Paul.D.Thomas@ENRON.com>","Fri, 29 Jun 2001 08:39:30 -0500",RE: Intranet Site,Rika r these new?\n\n -----Original Message---...,0,['http://eastpower.dev.corp.enron.com/summary/...
3,"""Thorne, Judy"" <Judy.Thorne@ENRON.com>","""Grass, John"" <John.Grass@ENRON.com>, ""Nemec, ...","Fri, 29 Jun 2001 10:35:17 -0500",FW: ENA Upstream Company information,"John/Gerald,\n\nWe are currently trading under...",0,[]
4,"""Williams, Jason R (Credit)"" <Jason.R.Williams...","""Nemec, Gerald"" <Gerald.Nemec@ENRON.com>, ""Dic...","Fri, 29 Jun 2001 10:40:02 -0500",New Master Physical,Gerald and Stacy -\n\nAttached is a worksheet ...,0,[]


In [17]:
df = pd.read_csv("../data/Nazario_5.csv")
df.head()

,sender,receiver,date,subject,body,label,urls
0,"""Hu, Sylvia"" <Sylvia.Hu@ENRON.com>","""Acevedo, Felecia"" <Felecia.Acevedo@ENRON.com>...","Fri, 29 Jun 2001 08:36:09 -0500","FW: June 29 -- BNA, Inc. Daily Labor Report",User ID: enrondlr\nPW: bnaweb22\n\n\n ...,0,"['http://web.bna.com', 'http://pubs.bna.com/ip..."
1,"""Webb, Jay"" <Jay.Webb@ENRON.com>","""Lambie, Chris"" <Chris.Lambie@ENRON.com>","Fri, 29 Jun 2001 09:37:04 -0500",NGX failover plan.,"\nHi Chris, \n\nTonight we are rolling out a ...",0,[]
2,"""Symms, Mark"" <Mark.Symms@ENRON.com>","""Thomas, Paul D."" <Paul.D.Thomas@ENRON.com>","Fri, 29 Jun 2001 08:39:30 -0500",RE: Intranet Site,Rika r these new?\n\n -----Original Message---...,0,['http://eastpower.dev.corp.enron.com/summary/...
3,"""Thorne, Judy"" <Judy.Thorne@ENRON.com>","""Grass, John"" <John.Grass@ENRON.com>, ""Nemec, ...","Fri, 29 Jun 2001 10:35:17 -0500",FW: ENA Upstream Company information,"John/Gerald,\n\nWe are currently trading under...",0,[]
4,"""Williams, Jason R (Credit)"" <Jason.R.Williams...","""Nemec, Gerald"" <Gerald.Nemec@ENRON.com>, ""Dic...","Fri, 29 Jun 2001 10:40:02 -0500",New Master Physical,Gerald and Stacy -\n\nAttached is a worksheet ...,0,[]


In [18]:
import pandas as pd
import numpy as np
import re

df = pd.read_csv("../data/Nazario_5.csv")
df.head()

,sender,receiver,date,subject,body,label,urls
0,"""Hu, Sylvia"" <Sylvia.Hu@ENRON.com>","""Acevedo, Felecia"" <Felecia.Acevedo@ENRON.com>...","Fri, 29 Jun 2001 08:36:09 -0500","FW: June 29 -- BNA, Inc. Daily Labor Report",User ID: enrondlr\nPW: bnaweb22\n\n\n ...,0,"['http://web.bna.com', 'http://pubs.bna.com/ip..."
1,"""Webb, Jay"" <Jay.Webb@ENRON.com>","""Lambie, Chris"" <Chris.Lambie@ENRON.com>","Fri, 29 Jun 2001 09:37:04 -0500",NGX failover plan.,"\nHi Chris, \n\nTonight we are rolling out a ...",0,[]
2,"""Symms, Mark"" <Mark.Symms@ENRON.com>","""Thomas, Paul D."" <Paul.D.Thomas@ENRON.com>","Fri, 29 Jun 2001 08:39:30 -0500",RE: Intranet Site,Rika r these new?\n\n -----Original Message---...,0,['http://eastpower.dev.corp.enron.com/summary/...
3,"""Thorne, Judy"" <Judy.Thorne@ENRON.com>","""Grass, John"" <John.Grass@ENRON.com>, ""Nemec, ...","Fri, 29 Jun 2001 10:35:17 -0500",FW: ENA Upstream Company information,"John/Gerald,\n\nWe are currently trading under...",0,[]
4,"""Williams, Jason R (Credit)"" <Jason.R.Williams...","""Nemec, Gerald"" <Gerald.Nemec@ENRON.com>, ""Dic...","Fri, 29 Jun 2001 10:40:02 -0500",New Master Physical,Gerald and Stacy -\n\nAttached is a worksheet ...,0,[]


In [19]:
def count_urls(url_string):
    try:
        urls = eval(url_string)
        return len(urls)
    except:
        return 0

df["num_urls"] = df["urls"].apply(count_urls)
df[["urls", "num_urls"]].head()

,urls,num_urls
0,"['http://web.bna.com', 'http://pubs.bna.com/ip...",41
1,[],0
2,['http://eastpower.dev.corp.enron.com/summary/...,3
3,[],0
4,[],0


In [20]:
def extract_domain(sender):
    match = re.search(r'@([^>]+)>?', str(sender))
    if match:
        return match.group(1).lower()
    return None

df["sender_domain"] = df["sender"].apply(extract_domain)
df[["sender", "sender_domain"]].head()

,sender,sender_domain
0,"""Hu, Sylvia"" <Sylvia.Hu@ENRON.com>",enron.com
1,"""Webb, Jay"" <Jay.Webb@ENRON.com>",enron.com
2,"""Symms, Mark"" <Mark.Symms@ENRON.com>",enron.com
3,"""Thorne, Judy"" <Judy.Thorne@ENRON.com>",enron.com
4,"""Williams, Jason R (Credit)"" <Jason.R.Williams...",enron.com


In [21]:
def extract_tld(domain):
    if domain is None:
        return None
    parts = domain.split('.')
    if len(parts) >= 2:
        return parts[-1]
    return None

df["tld"] = df["sender_domain"].apply(extract_tld)
df[["sender_domain", "tld"]].head()

,sender_domain,tld
0,enron.com,com
1,enron.com,com
2,enron.com,com
3,enron.com,com
4,enron.com,com


In [22]:
suspicious_tlds = {
    "ru","cn","br","xyz","top","click","link","rest",
    "monster","buzz","tk","to","ga","ml","cf"
}

def is_suspicious(tld):
    if tld is None:
        return 0
    return 1 if tld in suspicious_tlds else 0

df["suspicious_tld"] = df["tld"].apply(is_suspicious)
df[["sender_domain", "tld", "suspicious_tld"]].head()

,sender_domain,tld,suspicious_tld
0,enron.com,com,0
1,enron.com,com,0
2,enron.com,com,0
3,enron.com,com,0
4,enron.com,com,0


In [23]:
def extract_display_name(sender):
    if sender is None:
        return None
    sender = str(sender)
    if "<" in sender:
        return sender.split("<")[0].strip().strip('"').lower()
    return sender.lower()

df["display_name"] = df["sender"].apply(extract_display_name)
df[["sender", "display_name"]].head()

,sender,display_name
0,"""Hu, Sylvia"" <Sylvia.Hu@ENRON.com>","hu, sylvia"
1,"""Webb, Jay"" <Jay.Webb@ENRON.com>","webb, jay"
2,"""Symms, Mark"" <Mark.Symms@ENRON.com>","symms, mark"
3,"""Thorne, Judy"" <Judy.Thorne@ENRON.com>","thorne, judy"
4,"""Williams, Jason R (Credit)"" <Jason.R.Williams...","williams, jason r (credit)"


In [24]:
brand_keywords = ["bank", "paypal", "amazon", "apple", "wells", "chase", "support"]

def brand_domain_mismatch(row):
    name = row["display_name"]
    domain = row["sender_domain"]

    if name is None or domain is None:
        return 0

    for brand in brand_keywords:
        if brand in name:
            if brand not in domain:
                return 1  # mismatch
    return 0

df["brand_mismatch"] = df.apply(brand_domain_mismatch, axis=1)
df[["display_name", "sender_domain", "brand_mismatch"]].head()

,display_name,sender_domain,brand_mismatch
0,"hu, sylvia",enron.com,0
1,"webb, jay",enron.com,0
2,"symms, mark",enron.com,0
3,"thorne, judy",enron.com,0
4,"williams, jason r (credit)",enron.com,0


In [25]:
df[["num_urls", "suspicious_tld", "brand_mismatch", "label"]].describe()

,num_urls,suspicious_tld,brand_mismatch,label
count,3065.000000,3065.000000,3065.000000,3065.000000
mean,1.050897,0.020228,0.068842,0.510604
std,9.702091,0.140804,0.253226,0.499969
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,1.000000
75%,0.000000,0.000000,0.000000,1.000000
max,467.000000,1.000000,1.000000,1.000000


In [26]:
from sklearn.model_selection import train_test_split

features = df[["num_urls", "suspicious_tld", "brand_mismatch"]]
labels = df["label"]

X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.2, random_state=42
)

X_train.shape, X_test.shape

ModuleNotFoundError: No module named 'sklearn'

In [27]:
!python3 -m pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 11.1 MB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 308 kB 17.9 MB/s eta 0:00:01
     |████████████████████████████████| 30.3 MB 1.1 MB/s eta 0:00:011
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [28]:
from sklearn.model_selection import train_test_split

features = df[["num_urls", "suspicious_tld", "brand_mismatch"]]
labels = df["label"]

X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.2, random_state=42
)

X_train.shape, X_test.shape

((2452, 3), (613, 3))

In [29]:
X_train.shape, X_test.shape

((2452, 3), (613, 3))

In [30]:
from sklearn.linear_model import LogisticRegression

log_model = LogisticRegression()
log_model.fit(X_train, y_train)

log_train_acc = log_model.score(X_train, y_train)
log_test_acc = log_model.score(X_test, y_test)

log_train_acc, log_test_acc

(0.7589722675367048, 0.7275693311582382)

In [31]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=300, random_state=42)
rf.fit(X_train, y_train)

rf_train_acc = rf.score(X_train, y_train)
rf_test_acc = rf.score(X_test, y_test)

rf_train_acc, rf_test_acc

(0.7589722675367048, 0.7275693311582382)

In [32]:
from sklearn.metrics import confusion_matrix, classification_report

y_pred = rf.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[146 167]
 [  0 300]]
              precision    recall  f1-score   support

           0       1.00      0.47      0.64       313
           1       0.64      1.00      0.78       300

    accuracy                           0.73       613
   macro avg       0.82      0.73      0.71       613
weighted avg       0.82      0.73      0.71       613



In [33]:
def predict_email(sender, urls_list, display_name=None):
    
    # auto-extract domain + tld
    domain = extract_domain(sender)
    tld = extract_tld(domain)
    
    # auto-extract display name
    if display_name is None:
        name = extract_display_name(sender)
    else:
        name = display_name.lower()
    
    # compute features
    num_urls = len(urls_list)
    suspicious = 1 if tld in suspicious_tlds else 0

    # brand mismatch
    b_mismatch = 0
    for brand in brand_keywords:
        if name and brand in name and brand not in (domain or ""):
            b_mismatch = 1
    
    # build feature array
    sample = np.array([[num_urls, suspicious, b_mismatch]])
    
    # model prediction using random forest
    pred = rf.predict(sample)[0]
    score = rf.predict_proba(sample)[0][1]  # phishing probability
    
    return pred, score

In [34]:
predict_email(
    sender='"PayPal Support" <alerts@random.ru>',
    urls_list=["http://scam.ru/verify", "http://paypal.com.fake/login"]
)

/Users/inesnix/Library/Python/3.9/lib/python/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/inesnix/Library/Python/3.9/lib/python/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


(np.int64(0), np.float64(0.49666666666666665))

In [35]:
sample = pd.DataFrame([{
    "num_urls": num_urls,
    "suspicious_tld": suspicious,
    "brand_mismatch": b_mismatch
}])

pred = rf.predict(sample)[0]
score = rf.predict_proba(sample)[0][1]
return pred, score

NameError: name 'num_urls' is not defined

In [36]:
def predict_email(sender, urls_list):
    # --- extract number of URLs ---
    num_urls = len(urls_list)

    # --- extract sender domain ---
    import re
    match = re.search(r'@([^>]+)>?', str(sender))
    if match:
        domain = match.group(1).lower()
    else:
        domain = ""

    # --- suspicious TLD check ---
    tld = domain.split(".")[-1] if "." in domain else ""
    suspicious_tlds = {"ru","cn","br","xyz","top","click","link","rest","monster","buzz"}
    suspicious = 1 if tld in suspicious_tlds else 0

    # --- brand mismatch check ---
    display_name = str(sender).split("<")[0].strip().strip('"').lower()
    brand_keywords = ["paypal", "amazon", "apple", "bank", "enron"]
    b_mismatch = 0
    for brand in brand_keywords:
        if brand in display_name and brand not in domain:
            b_mismatch = 1

    # --- build sample row (IMPORTANT: use DataFrame not array) ---
    sample = pd.DataFrame([{
        "num_urls": num_urls,
        "suspicious_tld": suspicious,
        "brand_mismatch": b_mismatch
    }])

    # --- run model ---
    pred = rf.predict(sample)[0]
    score = rf.predict_proba(sample)[0][1]

    return pred, score

In [37]:
predict_email(
    sender='"PayPal Support" <alerts@random.ru>',
    urls_list=["http://scam.ru/verify", "http://paypal.com.fake/login"]
)

(np.int64(0), np.float64(0.49666666666666665))

In [38]:
predict_email(
    sender='"Christine" <christine@rwustl.edu>',
    urls_list=["https://en.wikipedia.org/wiki/Christine_(1983_film)", "https://en.wikipedia.org/wiki/Christine_(1983_film)"]
)

(np.int64(0), np.float64(0.0))

In [39]:
# Add this at the end of your notebook after training the Random Forest
import pickle

# Save the trained model
with open('../models/trained_model.pkl', 'wb') as f:
    pickle.dump(rf, f)

print("Model saved to models/trained_model.pkl")

FileNotFoundError: [Errno 2] No such file or directory: '../models/trained_model.pkl'

In [40]:
import os

# Create the models directory if it doesn't exist
os.makedirs('../models', exist_ok=True)
print("✓ models/ directory created")

✓ models/ directory created


In [41]:
import pickle

# Save the trained model
with open('../models/trained_model.pkl', 'wb') as f:
    pickle.dump(rf, f)

print("Model saved to models/trained_model.pkl")
```

SyntaxError: invalid syntax (2390386715.py, line 8)

In [42]:
import pickle

# Save the trained model
with open('../models/trained_model.pkl', 'wb') as f:
    pickle.dump(rf, f)

print("Model saved to models/trained_model.pkl")


Model saved to models/trained_model.pkl
